In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv

In [2]:
#Flying time
FlightHours = pd.read_csv('Flying _Time_China.csv')

print(FlightHours.head())
print(FlightHours.shape)

        city  Flying Time
0    Beijing        5.750
1   Changsha        4.167
2    Chengdu        4.300
3  Chongqing        4.200
4  Guangzhou        3.600
(24, 2)


In [3]:
# Minimum Flying Frequency
# to be updated
MinFreq = pd.read_csv('Min_Frequency_Flights.csv')
print(MinFreq.head())
print(MinFreq.shape)

        city  min_frequency
0    Beijing              2
1   Changsha              1
2    Chengdu              2
3  Chongqing              2
4  Guangzhou              2
(24, 2)


In [4]:
# Passenger Daily Demand
PassDem = pd.read_excel('china_city_passenger_demand.xlsx')

PassDem['daily_dem_target'] = PassDem['daily_dem']
PassDem['var_day_target'] = PassDem['var_day']

print(PassDem.head())
print(PassDem.shape)

        city  daily_dem      var_day  daily_dem_target  var_day_target
0    Beijing        896  7747.296296               896     7747.296296
1   Changsha        310   929.436117               310      929.436117
2    Chengdu        691  4603.084340               691     4603.084340
3  Chongqing        573  3171.978833               573     3171.978833
4  Guangzhou        625  3770.959789               625     3770.959789
(24, 5)


In [5]:
#Cost per Flight hour
CostPerFlightHour = pd.read_excel('AvgOpCost.xlsx')
print(CostPerFlightHour)
print(CostPerFlightHour.shape)

     Aircraft Type  Avg. Cost. Per Flight Hour in USD - Sep'19
0  Airbus A320 Neo                                      4378.0
1      Airbus A320                                      4378.0
2  Airbus A321 Neo                                      5149.0
3     Boeing 787-8                                      8213.0
4     Boeing 787-9                                      8973.5
(5, 2)


In [6]:
# Seat Capacity data
SeatCapacity = pd.read_csv('Seat Capacity.csv')
print(SeatCapacity)
print(SeatCapacity.shape)

     Aircraft Type  Seat Capacity
0  Airbus A320 Neo            194
1      Airbus A320            180
2  Airbus A321 Neo            244
3     Boeing 787-8            242
4     Boeing 787-9            296
(5, 2)


In [7]:
#Available FLight Operating Hours
AFOH = pd.read_csv('Fleet.csv')

#Assuming that an aircraft will be used 20 hours in a day
AFOH['Available Operating Hours per day'] = 20 * AFOH['Fleet Count']
print(AFOH)
print(AFOH.shape)

          Aircraft  Fleet Count  Available Operating Hours per day
0  Airbus A320 Neo           10                                200
1      Airbus A320           10                                200
2  Airbus A321 Neo           10                                200
3     Boeing 787-8            2                                 40
4     Boeing 787-9            2                                 40
(5, 3)


In [8]:
from gurobipy import GRB, Model, quicksum
from numpy import array

In [9]:
#########Parameters Set-up############

C = tuple( float(i)for i in CostPerFlightHour.iloc[:, -1])
D = tuple( float(i)for i in FlightHours.iloc[:, -1])
SC = tuple( float(i)for i in SeatCapacity.iloc[:, -1])
PDD = tuple( float(i)for i in PassDem.iloc[:, -2])
MF = tuple( float(i)for i in MinFreq.iloc[:, -1])
OH = tuple( float(i)for i in AFOH.iloc[:, -1])

#From the matrix, extract the number of variables
# M is number of types of aircrafts whose frequency to each destination we are determining
# N is number of destinations
M, N = (5, 24)

# Linear Programming

In [10]:
#########Model Set-up###############

m = Model("Airlines")

# Creat variables
# addVars ( *indices, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="" )

# Add demand constraint 5 Xij Non-Negativity Constraint
X = m.addVars(M, N, lb=0.0, ub=GRB.INFINITY, vtype = GRB.CONTINUOUS, name="FlightFrequencyPerAircraftPerRoute")

# Set objective
m.setObjective(quicksum(C[i] * X[i,j] * D[j] for i in range(M) for j in range(N)), GRB.MINIMIZE)

# Add supply constraints 1 Passenger Daily Demand: 
m.addConstrs(( quicksum(X[i,j] * SC[i] for i in range(M)) >= PDD[j] for j in range(N)) , "PassengerDemand")

# Add supply constraints 2 Minimum Flight Frequency Requirement
m.addConstrs(( quicksum(X[i,j] for i in range(M)) >= MF[j] for j in range(N) ), 'MinFlightFrequency')

# Add supply constraints 3 Available Fleet Operating Hours Per Aircraft Fleet per day
m.addConstrs( (quicksum(X[i,j] * D[j] for j in range(N)) <= OH[i] for i in range(M)) , 'FlightOperatingHours')

#Supressing the optimization output - Do we need to do this
m.setParam( 'OutputFlag', False )

# Solving the model
m.optimize()

print('Obj:', m.objVal)

Restricted license - for non-production use only - expires 2023-10-25
Obj: 919253.4092837707


In [12]:
Aircrafts = list(SeatCapacity.iloc[:, 0])
destinations = list(MinFreq.iloc[:, 0])
output_lp = array([[0.00] * M] * N)
for j in range(N):
    for i in range(M):
        output_lp[j][i] = round(X[i,j].x,4)

print(output_lp)
DataFrame(array(output_lp), index = destinations , columns = Aircrafts)

[[0.     0.     3.6721 0.     0.    ]
 [0.     0.     1.2705 0.     0.    ]
 [0.     0.     2.832  0.     0.    ]
 [0.     0.     2.3484 0.     0.    ]
 [0.     0.     2.5615 0.     0.    ]
 [0.     0.     1.0779 0.     0.    ]
 [0.     0.     1.1148 0.     0.    ]
 [1.02   0.     0.98   0.     0.    ]
 [0.66   0.     0.34   0.     0.    ]
 [0.     0.     2.0492 0.     0.    ]
 [1.     0.     0.     0.     0.    ]
 [0.     0.     1.1189 0.     0.    ]
 [1.     0.     0.     0.     0.    ]
 [1.     0.     0.     0.     0.    ]
 [0.     0.     1.0205 0.     0.    ]
 [0.     0.     1.0574 0.     0.    ]
 [0.     0.     4.1639 0.     0.    ]
 [0.56   0.     0.44   0.     0.    ]
 [0.     0.     2.3156 0.     0.    ]
 [0.18   0.     0.82   0.     0.    ]
 [0.     0.     1.0738 0.     0.    ]
 [0.     0.     1.2582 0.     0.    ]
 [0.38   0.     1.62   0.     0.    ]
 [0.     0.     1.2049 0.     0.    ]]


,Airbus A320 Neo,Airbus A320,Airbus A321 Neo,Boeing 787-8,Boeing 787-9
Beijing,0.00,0.0,3.6721,0.0,0.0
Changsha,0.00,0.0,1.2705,0.0,0.0
Chengdu,0.00,0.0,2.8320,0.0,0.0
Chongqing,0.00,0.0,2.3484,0.0,0.0
Guangzhou,0.00,0.0,2.5615,0.0,0.0
Guiyang,0.00,0.0,1.0779,0.0,0.0
Haikou,0.00,0.0,1.1148,0.0,0.0
Hangzhuo,1.02,0.0,0.9800,0.0,0.0
Jinan,0.66,0.0,0.3400,0.0,0.0
Kunming,0.00,0.0,2.0492,0.0,0.0


In [14]:
output_p1_2 = array([[0] * M] * N)
for j in range(N):
    for i in range(M):
        output_p1_2[j][i] = round(output_lp[j][i], 0)
print(output_p1_2)

objVal = 0
for j in range(N):
    for i in range(M):
        objVal += output_p1_2[j][i] * C[i] * D[j]

print('Rounded of continuous:',objVal)

objVal = round(objVal, 4)
df_p1_2 = DataFrame(array(output_p1_2), index = destinations , columns = Aircrafts)
df_p1_2

[[0 0 4 0 0]
 [0 0 1 0 0]
 [0 0 3 0 0]
 [0 0 2 0 0]
 [0 0 3 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [1 0 1 0 0]
 [1 0 0 0 0]
 [0 0 2 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 4 0 0]
 [1 0 0 0 0]
 [0 0 2 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 2 0 0]
 [0 0 1 0 0]]
Rounded of continuous: 894028.387


,Airbus A320 Neo,Airbus A320,Airbus A321 Neo,Boeing 787-8,Boeing 787-9
Beijing,0,0,4,0,0
Changsha,0,0,1,0,0
Chengdu,0,0,3,0,0
Chongqing,0,0,2,0,0
Guangzhou,0,0,3,0,0
Guiyang,0,0,1,0,0
Haikou,0,0,1,0,0
Hangzhuo,1,0,1,0,0
Jinan,1,0,0,0,0
Kunming,0,0,2,0,0


# Integer Programming

In [15]:
#########Model Set-up###############

m = Model("Airlines")

# Creat variables
# addVars ( *indices, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="" )

# Add demand constraint 5 Xij Non-Negativity Constraint
X = m.addVars(M, N, lb=0.0, ub=GRB.INFINITY, vtype = GRB.INTEGER, name="FlightFrequencyPerAircraftPerRoute")

# Set objective
m.setObjective(quicksum(C[i] * X[i,j] * D[j] for i in range(M) for j in range(N)), GRB.MINIMIZE)

# Add supply constraints 1 Passenger Daily Demand: 
m.addConstrs(( quicksum(X[i,j] * SC[i] for i in range(M)) >= PDD[j] for j in range(N)) , "PassengerDemand")

# Add supply constraints 2 Minimum Flight Frequency Requirement
m.addConstrs(( quicksum(X[i,j] for i in range(M)) >= MF[j] for j in range(N) ), 'MinFlightFrequency')

# Add supply constraints 3 Available Fleet Operating Hours Per Aircraft Fleet per day
m.addConstrs( (quicksum(X[i,j] * D[j] for j in range(N)) <= OH[i] for i in range(M)) , 'FlightOperatingHours')

#Supressing the optimization output - Do we need to do this
m.setParam( 'OutputFlag', False )

# Solving the model
m.optimize()

print('Obj:', m.objVal)

Obj: 1079439.6630000002


In [16]:
output_ip = array([[0] * M] * N)
for j in range(N):
    for i in range(M):
        output_ip[j][i] = X[i,j].x

print(output_ip)
DataFrame(array(output_ip), index = destinations , columns = Aircrafts)

[[1 0 3 0 0]
 [2 0 0 0 0]
 [0 0 3 0 0]
 [3 0 0 0 0]
 [2 0 1 0 0]
 [2 0 0 0 0]
 [2 0 0 0 0]
 [1 0 1 0 0]
 [0 0 1 0 0]
 [0 3 0 0 0]
 [1 0 0 0 0]
 [2 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [2 0 0 0 0]
 [2 0 0 0 0]
 [4 0 1 0 0]
 [0 0 1 0 0]
 [3 0 0 0 0]
 [0 0 1 0 0]
 [2 0 0 0 0]
 [2 0 0 0 0]
 [0 0 2 0 0]
 [2 0 0 0 0]]


,Airbus A320 Neo,Airbus A320,Airbus A321 Neo,Boeing 787-8,Boeing 787-9
Beijing,1,0,3,0,0
Changsha,2,0,0,0,0
Chengdu,0,0,3,0,0
Chongqing,3,0,0,0,0
Guangzhou,2,0,1,0,0
Guiyang,2,0,0,0,0
Haikou,2,0,0,0,0
Hangzhuo,1,0,1,0,0
Jinan,0,0,1,0,0
Kunming,0,3,0,0,0


### Determining frequency by averaging across samples

In [17]:
from numpy import maximum, random
import random

In [18]:
#From the matrix, extract the number of variables
M, N = (5, 24)
x_1 = [0] * N
x_2 = [0] * N
x_3 = [0] * N
x_4 = [0] * N
x_5 = [0] * N
Sample_Size = 200
objVal = 0

#Parameters Setup
C = tuple( float(i)for i in CostPerFlightHour.iloc[:, -1])
D = tuple( float(i)for i in FlightHours.iloc[:, -1])
SC = tuple( float(i)for i in SeatCapacity.iloc[:, -1])
MF = tuple( float(i)for i in MinFreq.iloc[:, -1])
OH = tuple( float(i)for i in AFOH.iloc[:, -1])

for k in range(Sample_Size):

    ######### Changing Demand ############
    PDD = tuple( float(i)for i in PassDem.iloc[:, -2])
    demand = np.array(PDD)
    
    #Considering covariance
    var = tuple( float(i) for i in PassDem.iloc[:, -1])
    cov = np.array(var)
    sigma = np.sqrt(cov)
        
    for i in range(len(PDD)):
        demand[i] = random.normalvariate(PDD[i], sigma[i])
    
    #########Model Set-up###############
    m = Model("Airlines")
    
    # Add demand constraint 5 Xij Non-Negativity Constraint
    X = m.addVars(M, N, vtype = GRB.INTEGER, name="FlightFrequencyPerAircraftPerRoute")

    # Set objective
    m.setObjective(quicksum(C[i] * X[i,j] * D[j] for i in range(M) for j in range(N)), GRB.MINIMIZE)

    # Add supply constraints 1 Passenger Daily Demand: 
    m.addConstrs(( quicksum(X[i,j] * SC[i] for i in range(M)) >= demand[j] for j in range(N)) , "PassengerDemand")

    # Add supply constraints 2 Minimum Flight Frequency Requirement
    m.addConstrs(( quicksum(X[i,j] for i in range(M)) >= MF[j] for j in range(N) ), 'MinFlightFrequency')

    # Add supply constraints 3 Available Fleet Operating Hours Per Aircraft Fleet per day
    m.addConstrs( (quicksum(X[i,j] * D[j] for j in range(N)) <= OH[i] for i in range(M)) , 'FlightOperatingHours')

    #Supressing the optimization output - Do we need to do this
    m.setParam( 'OutputFlag', False )

    # Solving the model
    m.optimize()

    # Print optimal solutions and optimal value
    objVal += m.objVal
    
    for j in range(N):
        for i in range(M):
            if i + 1 == 1:
                x_1[j] += X[i,j].x
            if i + 1 == 2:
                x_2[j] += X[i,j].x
            if i + 1 == 3:
                x_3[j] += X[i,j].x
            if i + 1 == 4:
                x_4[j] += X[i,j].x
            if i + 1 == 5:
                x_5[j] += X[i,j].x

objVal = objVal / Sample_Size
x_1 = np.round_(np.array(x_1) / Sample_Size)
x_2 = np.round_(np.array(x_2) / Sample_Size)
x_3 = np.round_(np.array(x_3) / Sample_Size)
x_4 = np.round_(np.array(x_4) / Sample_Size)
x_5 = np.round_(np.array(x_5) / Sample_Size)

print(objVal)

1068754.8995049996


In [19]:
tble = pd.DataFrame(np.column_stack( (x_1, x_2, x_3, x_4, x_5) ), index = destinations , columns = Aircrafts)
tble['Total'] = tble[Aircrafts].sum(axis = 1)
tble

,Airbus A320 Neo,Airbus A320,Airbus A321 Neo,Boeing 787-8,Boeing 787-9,Total
Beijing,2.0,0.0,2.0,0.0,0.0,4.0
Changsha,1.0,1.0,0.0,0.0,0.0,2.0
Chengdu,1.0,0.0,2.0,0.0,0.0,3.0
Chongqing,2.0,0.0,1.0,0.0,0.0,3.0
Guangzhou,1.0,0.0,1.0,0.0,0.0,2.0
Guiyang,1.0,0.0,0.0,0.0,0.0,1.0
Haikou,1.0,0.0,0.0,0.0,0.0,1.0
Hangzhuo,1.0,0.0,1.0,0.0,0.0,2.0
Jinan,0.0,0.0,1.0,0.0,0.0,1.0
Kunming,1.0,1.0,1.0,0.0,0.0,3.0
